# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place
- **Interrupts break the flow** - Human-in-the-loop requires manual resumption

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss
- **Interactive interrupt handling** - Automatic prompts and agent resumption with `run()`

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [3]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='What files are in the current directory?', additional_kwargs={}, response_metadata={}, id='6032e894-e8e5-4882-a123-fffee0730ae6')])}}
---
{'SummarizationMiddleware.before_model': None}
---{'model': {'messages': [AIMessage(content=[{'text': "I'll list the files in the current directory for you.", 'type': 'text'}, {'id': 'toolu_019ifLxGo51EssTxhscxURuB', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01UyW3e5Meaz2oTUSkY2bn5g', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 6258}, 'cache_creation_input_tokens': 6258, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 64, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'an

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [3]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [4]:
display = JupyterDisplay()

display.run(
    graph=agent,
    input_data=dict(messages=["What files are in the current directory?"]),
    config=dict(configurable=dict(thread_id="display-demo")),
)

OK ls path=/ 36ms

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ I'll list the files in the current directory for you.The current directory   │
│ contains:                                                                    │
│                                                                              │
│ 1. **/.ipynb_checkpoints/** - Directory for Jupyter notebook checkpoints     │
│ 2. **/__pycache__/** - Directory for Python bytecode cache                   │
│ 3. **/agent.py** - A Python file                                             │
│ 4. **/jupyter_display.py** - A Python file                                   │
│ 5. **/jupyter_example.ipynb** - A Jupyter notebook file                      │
│                                                                              │
│ Would you like me to explore any of these files or directories in more       │
│ detail?                                                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

done

---
## Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel and an input prompt. Type your decision and the agent continues automatically:

In [5]:
# Using run() - handles interrupts interactively!
display = JupyterDisplay()

display.run(
    graph=agent,
    input_data=dict(messages=["what's in the third file of this folder? Use think_tool, write_todos to work this out. Use the bash tool for ls"]),
    config=dict(configurable=dict(thread_id="interactive-demo")),
)
# When an interrupt occurs, you'll be prompted: "Decision (approve/edit/reject): "
# Type your choice and the agent continues automatically!

OK think_tool reflection=The user wants me to:
1. L... 14ms

reflection: The user wants me to:
1. List files in the current folder
2. Identify the third file
3. Show its contents

They specifically requested I use:
- think_tool (which I'm using now)
- write_todos (to plan ...

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ I'll help you find out what's in the third file of this folder. Let me start │
│ by thinking through this task and creating a plan.                           │
╰──────────────────────────────────────────────────────────────────────────────╯

OK write_todos todos=[{'content': 'Use bash ls c... 18ms

todos: 
  ▶ Use bash ls command to list files in the current folder
  ○ Identify the third file from the list
  ○ Read and display the contents of the third file

...

OK read_file file_path=/jupyter_display.py, limit=-1 11ms

OK write_todos todos=[{'content': 'Use bash ls c... 36ms

todos: 
  ✓ Use bash ls command to list files in the current folder
  ✓ Identify the third file from the list
  ✓ Read and display the contents of the third file

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ Perfect! I've completed the task. Here's what I found:                       │
│                                                                              │
│ ## Summary                                                                   │
│                                                                              │
│ The **third file** in the folder is **`jupyter_display.py`**.                │
│                                                                              │
│ ### Contents:                                                                │
│ This is a Python example file (265 lines) that demonstrates how to use the   │
│ **JupyterDisplay adapter** from the LangGraph Stream Parser library. The     │
│ file shows how to create rich, real-time visualizations of LangGraph agent   │
│ output in Jupyter notebooks.                                                 │
│                                                                              │
│ **Key sections include:**                                                    │
│ 1. **Basic Usage** - Simple streaming with the `stream()` method             │
│ 2. **Manual...                                                               │
╰──────────────────────────────────────────────────────────────────────────────╯

done

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `...` (yellow) - Running
- `OK` (green) - Success (with execution time)
- `ERR` (red) - Error

### 2. Interactive Interrupt Handling

When using `run()`, interrupts are handled automatically:
1. A prominent red panel shows the tool requesting approval
2. You're prompted for a decision: `Decision (approve/edit/reject):`
3. The agent resumes with your choice - no manual intervention needed!

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

### 4. Todo List Visualization

When agents use todo/task tracking tools, tasks are displayed with status icons:
- `✓` (green) - Completed
- `▶` (yellow) - In progress  
- `○` (dim) - Pending

---
## Custom Event Processing

For advanced use cases (filtering events, custom display logic), use `StreamParser` directly:

In [ ]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    # Custom filtering example: skip certain events
    if isinstance(event, ToolCallStartEvent) and event.name == "ls":
        continue  # Skip ls tool display
    
    display.update(event)
    
    # Custom handling example
    if isinstance(event, InterruptEvent):
        print(f"\n[Custom handler: {len(event.action_requests)} action(s) need approval]")

---
## Configuration Options

In [9]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (`...` → `OK`/`ERR`) |
| Interrupts stop execution | Interactive prompts with auto-resume |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |

## Quick Start

```python
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

display = JupyterDisplay()

display.run(
    graph=agent,
    input_data={"messages": [("user", "Your prompt")]},
    config={"configurable": {"thread_id": "my-session"}}
)
```

For custom event processing (filtering, transformation), use `StreamParser.parse()` with `display.update()`.